In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime
#After restarting runtime, you don't need to run this cell again.

Install Detectron2 for generating 2D poses.

In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
%cd /content/drive/My Drive/videopose/VideoPose3D/inference

/content/drive/My Drive/videopose/VideoPose3D/inference


preprocessing video for generating 2D data
  * get videos from inference/inputvideos
  * export the result into inference/prevideos

In [5]:
!python infer_video_d2.py \
    --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml \
    --output-dir pre_videos \
    --image-ext mp4 \
    inputvideos

Processing inputvideos/S001C002P003R002A023.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx26

copy files for generating 2D data

In [6]:
from distutils.dir_util import copy_tree
copy_tree("/content/drive/My Drive/videopose/VideoPose3D/inference/pre_videos","/content/drive/My Drive/videopose/VideoPose3D/data/input")

['/content/drive/My Drive/videopose/VideoPose3D/data/input/S001C002P003R002A023.mp4.npz',
 '/content/drive/My Drive/videopose/VideoPose3D/data/input/S001C002P003R002A001.mp4.npz']

In [101]:
%cd '/content/drive/My Drive/videopose/VideoPose3D/data'

/content/drive/My Drive/videopose/VideoPose3D/data


generating 2D poses
  * -i: input file
  * -o: output file

In [102]:
!python prepare_data_2d_custom.py -i input -o twoDposes

Parsing 2D detections from input
Processing input/S001C002P003R002A023.mp4.npz
56 total frames processed
0 frames were interpolated
----------
Processing input/S001C002P003R002A001.mp4.npz
12 total frames processed
0 frames were interpolated
----------
Saving...
Done.


In [103]:
%cd '/content/drive/My Drive/videopose/VideoPose3D'

/content/drive/My Drive/videopose/VideoPose3D


In [104]:
!python run.py -d custom \
                -k twoDposes \
                -arc 3,3,3,3,3 \
                -c checkpoint \
                --evaluate pretrained_h36m_detectron_coco.bin \
                --render \
                --viz-action custom \
                --viz-camera 0 
                #--viz-video inference/inputvideos/ mp4 \
                #--viz-export '/content/drive/My Drive/videopose/VideoPose3D/final_output'
                #--viz-output drinking.mp4 \
                #--viz-size 6
                #--viz-subject inference/inputvideos/ mp4 \

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='twoDposes', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=5, viz_skip=0, viz_subject=None, viz_video=None, warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
S001C002P003R002A023.mp4
INFO: Receptive field: 243 frames
INFO: Traina

In [108]:
from distutils.dir_util import copy_tree
copy_tree("/content/drive/My Drive/videopose/VideoPose3D/final_output","/content/drive/My Drive/MS-G3D/data/nturgbd_raw/nturgb+d_skeletons120")

['/content/drive/My Drive/MS-G3D/data/nturgbd_raw/nturgb+d_skeletons120/S001C002P003R002A023.skeleton',
 '/content/drive/My Drive/MS-G3D/data/nturgbd_raw/nturgb+d_skeletons120/S001C002P003R002A001.skeleton']

MS-G3D

In [121]:
%cd '/content/drive/My Drive/MS-G3D/data_gen'

/content/drive/My Drive/MS-G3D/data_gen


In [122]:
!python3 ntu120_gendata.py

xsub train
0it [00:00, ?it/s]
pad the null frames with the previous frames
0it [00:00, ?it/s]
sub the center joint #1 (spine joint in ntu and neck joint in kinetics)
0it [00:00, ?it/s]
parallel the bone between hip(jpt 0) and spine(jpt 1) of the first person to the z axis
0it [00:00, ?it/s]
parallel the bone between right shoulder(jpt 8) and left shoulder(jpt 4) of the first person to the x axis
0it [00:00, ?it/s]
xsub val
100% 2/2 [00:00<00:00, 204.88it/s]
pad the null frames with the previous frames
100% 2/2 [00:00<00:00, 2089.32it/s]
sub the center joint #1 (spine joint in ntu and neck joint in kinetics)
100% 2/2 [00:00<00:00, 3377.06it/s]
parallel the bone between hip(jpt 0) and spine(jpt 1) of the first person to the z axis
100% 2/2 [00:00<00:00, 45.50it/s]
parallel the bone between right shoulder(jpt 8) and left shoulder(jpt 4) of the first person to the x axis
100% 2/2 [00:00<00:00, 45.72it/s]
xset train
0it [00:00, ?it/s]
pad the null frames with the previous frames
0it [00:00,

In [123]:
!python gen_bone_data.py --dataset ntu120

ntu120/xset train
100% 25/25 [00:00<00:00, 53038.75it/s]
ntu120/xset val
100% 25/25 [00:00<00:00, 8277.36it/s]
ntu120/xsub train
100% 25/25 [00:00<00:00, 50291.41it/s]
ntu120/xsub val
100% 25/25 [00:00<00:00, 8440.60it/s]


In [124]:
%cd '/content/drive/My Drive/MS-G3D/graph'

/content/drive/My Drive/MS-G3D/graph


In [125]:
!python ntu_rgb_d.py 

<Figure size 640x480 with 3 Axes>
(25, 25) (25, 25) (25, 25)


In [126]:
%cd '/content/drive/My Drive/MS-G3D/feeders'

/content/drive/My Drive/MS-G3D/feeders


In [127]:
!python feeder.py 

In [128]:
%cd '/content/drive/My Drive/MS-G3D/'

/content/drive/My Drive/MS-G3D


In [119]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 4.1 MB/s 


In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!python setup.py install

In [129]:
%cd '/content/drive/My Drive/MS-G3D/'

/content/drive/My Drive/MS-G3D


In [130]:
!bash eval_pretrained.sh

main.py:687: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  default_arg = yaml.load(f)
[ Mon Mar 21 23:55:40 2022 ] Model total number of params: 3194595
Cannot parse global_step from model weights filename
[ Mon Mar 21 23:55:40 2022 ] Loading weights from pretrained-models/ntu60-xsub-joint-fusion.pt
Traceback (most recent call last):
  File "/content/drive/MyDrive/MS-G3D/feeders/feeder.py", line 44, in load_data
    with open(self.label_path) as f:
FileNotFoundError: [Errno 2] No such file or directory: './data/ntu/xsub/val_label.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "main.py", line 702, in <module>
    main()
  File "main.py", line 697, in main
    processor = Processor(arg)
  File "main.py", line 253, in __init__
    self.load_data()
  File "main.py", line 398, in load_data
    dataset=Feed